# Experiments for paper on surrogate metric

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy.signal
import scipy.spatial.distance as dist
from simulation import WangStamatiadis, WSDriver, SimulationApproaching, ws_approaching_pars, \
    LeaderInteraction, IDMPlus, SimulationLongitudinal, IDMParameters, LeaderInteractionParameters
from stats import KDE, kde_from_file

### Parameters

In [ ]:
OVERWRITE = False

## Show that our metric is a generalisation of W&S' metric

In [ ]:
N_MONTE_CARLO_WS = 500
WS_TTCS = np.linspace(.5, 4, 36)
WS_SPEED_DIFFS = [10, 20, 30]
WS_LINESTYLES = ['-', '--', ':']
WS_COLORS = [(0, 0, 0), (.3, .3, .3)]
WS_LINEWIDTHS = [3, 2]
WS_FILENAME = os.path.join("data", "7_simulation_results", "ws_comparison.p")

In [ ]:
driver = WSDriver()
simulation = SimulationApproaching([driver], [ws_approaching_pars])
simulation.min_simulations = 30
def ws_with_our_method(ttc: float, speed_diff: float):
    return simulation.get_probability(dict(vego=speed_diff, ratio_vtar_vego=0, 
                                           init_position=ttc*speed_diff))
    
ws = WangStamatiadis()

In [ ]:
if os.path.exists(WS_FILENAME):
    with open(FILENAME, "rb") as file:
        result = pickle.load(file)
if not os.path.exists(WS_FILENAME) or OVERWRITE or \
        not result.shape == (len(WS_TTCS), len(WS_SPEED_DIFFS), 2):
    result = np.zeros((len(WS_TTCS), len(WS_SPEED_DIFFS), 2))
    np.random.seed(0)
    for i, ttc in enumerate(WS_TTCS):
        for j, speed_diff in enumerate(WS_SPEED_DIFFS):
            result[i, j, 0] = ws.prob_collision(ttc, speed_diff)
            result[i, j, 1] = ws_with_our_method(ttc, speed_diff)
    with open(WS_FILENAME, "wb") as file:
        pickle.dump(result, file)

In [ ]:
for i, speed_diff in enumerate(WS_SPEED_DIFFS):
    plt.plot(WS_TTCS, result[:, i, 0], ls=WS_LINESTYLES[i], color=WS_COLORS[0], 
             lw=WS_LINEWIDTHS[0])
    plt.plot(WS_TTCS, result[:, i, 1], ls=WS_LINESTYLES[i], color=WS_COLORS[1],
             lw=WS_LINEWIDTHS[1])
plt.xlabel("TTC [s]")
plt.ylabel("$P(C|x)$ according to the WS metric")

## Apply method on NGSIM data set to create a risk metric

In [ ]:
NGSIM_DATA = os.path.join("data", "8_interactions_v3", 
                          "dNGSIM_iLongitudinal_mFull_Reconstruction_e100_wi1_w1_hs32_bs2",
                          "interactions.pkl")
NGSIM_KDE = os.path.join("data", "6_kde", "NGSIM.p")
NGSIM_PROB_COLLISION = os.path.join("data", "7_simulation_results", 
                                    "prob_collision_NGSIM.csv")

In [ ]:
# Load data.
with open(NGSIM_DATA, 'rb') as file:
    all_interactions = pickle.load(file)
locations = sorted(all_interactions.keys())

In [ ]:
# Filter for speed and acceleration
def filter_signal(signal):
    return scipy.signal.savgol_filter(signal, 15, 1)

In [ ]:
def get_pars(vel_acc):
    if len(vel_acc) < 15:
        return np.zeros((0, 4))
    if np.max(np.abs(np.diff(vel_acc['Velocity_X']))) > 1.5:
        return np.zeros((0, 4))
    
    data = vel_acc.copy()
    data['ax_savgol'] = filter_signal(data["Acceleration_X"])
    data['vx_savgol'] = filter_signal(data["Velocity_X"])
    i = data.index[scipy.signal.find_peaks(-data['vx_savgol'], prominence=1)[0]]
    data['endspeed'] = np.nan
    data['endtime'] = np.nan
    data.loc[i, 'endspeed'] = data.loc[i, 'vx_savgol']
    data.loc[i, 'endtime'] = i
    data = data.fillna(method='backfill')
    data = data.dropna()

    data['duration'] = data['endtime'] - data.index
    data['vdiff'] = data['endspeed'] - data['vx_savgol']
    data['amean'] = data['vdiff'] / data['duration']
    data = data.drop(i)
    return data[['vx_savgol', 'ax_savgol', 'vdiff', 'amean']].values[::10]

In [ ]:
if OVERWRITE or not os.path.exists(NGSIM_KDE):
    parameters = []
    for location in locations:
        parameters += [get_pars(interaction['leader']) for interaction in 
                       all_interactions[location].values()]
    kde = KDE(np.concatenate(parameters), scaling=True)
    kde.clustering(kde._maxdist()*5)
    kde.compute_bandwidth()
    print("Bandwidth: {:.4f}".format(kde.get_bandwidth()))
    kde.pickle(NGSIM_KDE)
else:
    kde = kde_from_file(NGSIM_KDE)

In [ ]:
def leader_parameters(**kwargs):
    return LeaderInteractionParameters(init_position=kwargs["gap"],
                                       init_speed=kwargs["v0_lead"],
                                       init_acceleration=kwargs["a0_lead"],
                                       speed_difference=kwargs["dv"],
                                       duration=kwargs["duration"])

def follower_parameters(**kwargs):
    return IDMParameters(amin=kwargs["amin"],
                         speed=kwargs["v0_host"],
                         n_reaction=int(kwargs["tr"]*100),
                         init_speed=kwargs["v0_host"],
                         init_position=0)

def get_other_pars(**kwargs):
    # Get the speed difference and the mean acceleration from the KDE.
    while True:
        (kwargs["dv"], kwargs["amean"]), = kde.conditional_sample([0, 1], [kwargs["v0_lead"], 
                                                                           kwargs["a0_lead"]])
        if np.sign(kwargs["dv"]) == np.sign(kwargs["amean"]):
            break
    kwargs["duration"] = kwargs["dv"] / kwargs["amean"]
    
    # Get reaction time from a lognormal distribution with mean=.92, std=0.28
    kwargs["tr"] = np.random.lognormal(np.log(.92**2 / np.sqrt(.92**2 + .28**2)), 
                                       np.sqrt(np.log(1 + .28**2/.92**2)))
    
    # Get the braking capacity from a truncated normal distribution
    while True:
        kwargs["amin"] = np.random.normal(-8.45, 1.4)
        if -12.68 < kwargs["amin"] < -4.23:
            break
    
    return kwargs

simulation = SimulationLongitudinal(LeaderInteraction(), leader_parameters,
                                    IDMPlus(), follower_parameters)
simulation.min_simulation_time = 2

def get_probability(**kwargs):
    """
    Parameters to provide:
    - v0_lead
    - a0_lead
    - v0_host
    - gap
    """
    # If the host speed is zero, always return 0.0
    if "v0_host" in kwargs:
        if kwargs["v0_host"] <= 0.0:
            return 0.0
    
    min_sim = 10
    max_sim = 100
    results = np.zeros(max_sim)
    for i in range(max_sim):
        parameters = get_other_pars(**kwargs)
        results[i] = simulation.simulation(parameters)
        
        if i+1 >= min_sim:
            # If results are all the same, return either 0.0 or 1.0
            if np.std(results[:i+1]) < 1e-8:
                if results[0] > 0.0:
                    return 0.0
                return 1.0
            
            kde_result = KDE(results[:i+1], scaling=True)
            kde_result.compute_bandwidth()
            cdf_zero = kde_result.cdf(np.array([0.0]))[0]
            if np.sqrt(cdf_zero*(1-cdf_zero)/(i+1)) < 0.01:
                break
    
    if np.isnan(cdf_zero):
        asfdasdffd
    return cdf_zero

In [ ]:
# Create grid
def grid_pars(interaction):
    if len(interaction['leader']) < 15:
        return np.zeros((0, 4))
    interaction['leader']['ax_savgol'] = filter_signal(interaction['leader']["Acceleration_X"])
    interaction['leader']['vx_savgol'] = filter_signal(interaction['leader']["Velocity_X"])
    pars = pd.DataFrame(interaction["leader"][["vx_savgol", "ax_savgol"]].values,
                        columns=["v0_lead", "a0_lead"], index=interaction["leader"].index)
    interaction['follower']['vx_savgol'] = filter_signal(interaction['follower']["Velocity_X"])
    pars["v0_host"] = interaction["follower"]["vx_savgol"]
    gap = (interaction["leader"]["Position_X"] - interaction["follower"]["Position_X"] - 
           interaction["leader"]["Length"]/2 - interaction["follower"]["Length"]/2)
    gap[gap < 0] = np.exp(-5)  # Lower limit
    pars["loggap"] = np.log(gap)
    return pars

In [ ]:
if OVERWRITE or not os.path.exists(NGSIM_PROB_COLLISION):
    parameters = []
    for location in locations:
        parameters += [grid_pars(interaction) for interaction in 
                       all_interactions[location].values()]
    parameters = np.concatenate(parameters)
    
    grid = parameters.copy()
    scaling = [2, .5, 2, .25]
    grid[:, 0] = np.clip(grid[:, 0], 0, 100)
    # grid[:, 1] = np.clip(grid[:, 1], -5, 5)
    grid[:, 2] = np.clip(grid[:, 2], 0, 100)
    grid[:, 3] = np.clip(grid[:, 3], -5, 5)
    grid = np.round(grid / scaling)
    grid = np.unique(grid, axis=0)
    grid = grid * scaling
else:
    df = pd.read_csv(NGSIM_PROB_COLLISION)
    grid = df[["v0_lead", "a0_lead", "v0_host", "loggap"]].values

In [ ]:
# Evaluate the collision probability for the grid
def get_probability_grid_pars(row):
    return get_probability(v0_lead=row[0], a0_lead=row[1],
                           v0_host=row[2], gap=np.exp(row[3]))

In [ ]:
if OVERWRITE or not os.path.exists(NGSIM_PROB_COLLISION):
    prob_collision = [get_probability_grid_pars(row) for row in tqdm(grid)]
    df = pd.DataFrame(grid, columns=("v0_lead", "a0_lead", "v0_host", "loggap"))
    df["prob_collision"] = prob_collision
    df.to_csv(filename_prob_collision)
else:
    prob_collision = df["prob_collision"].values

In [ ]:
# Define function for the interpolation
scaling = np.std(grid, axis=0)
scaling[3] /= 5
grid_scaled = grid / scaling
def prob_ngsim(v0_lead, a0_lead, v0_host, gap):
    tmp = np.array([[v0_lead, a0_lead, v0_host, np.log(gap)]]) / scaling
    sq_distance = dist.cdist(grid_scaled, tmp, metric='sqeuclidean')
    weights = np.exp(-sq_distance / 2 / (0.3**2))  # Bandwidth of .3
    probability = np.dot(prob_collision, weights) / np.sum(weights, axis=0)
    #if probability > 0.3:
    #    print(tmp * scaling)
    #    print(grid_scaled[np.argmax(weights)] * scaling)
    #    aasffa
    return probability

## Try method for scenario 1: No risk

## Try method for scenario 2: Risky

In [ ]:
V0_LEADER = 20
DV_LEADER = 10
A_LEADER = 3
T_LEADER = 3

V0_HOST = 24
DV1_HOST = 16
DV2_HOST = 2
A1_HOST = 4
A2_HOST = 0.5
T_HOST = 4

D_INIT = 40
TMAX = 12

In [ ]:
time = np.arange(0, TMAX+0.01, 0.01)

v_lead = V0_LEADER * np.ones_like(time)
a_lead = np.zeros_like(time)
v_lead[time > DV_LEADER/A_LEADER+T_LEADER] = V0_LEADER - DV_LEADER
i = np.logical_and(time > T_LEADER, time <= DV_LEADER/A_LEADER+T_LEADER)
v_lead[i] = DV_LEADER/2*(np.cos(np.pi*(time[i]-T_LEADER)*A_LEADER/DV_LEADER)-1)+V0_LEADER
a_lead[i] = -DV_LEADER/2*np.sin(np.pi*(time[i]-T_LEADER)*A_LEADER/DV_LEADER)*np.pi*A_LEADER/DV_LEADER

v_host = V0_HOST * np.ones_like(time)
a_host = np.zeros_like(time)
t1 = DV1_HOST / A1_HOST + T_HOST
i = np.logical_and(time > T_HOST, time <= t1)
v_host[i] = DV1_HOST/2*(np.cos(np.pi*(time[i]-T_HOST)*A1_HOST/DV1_HOST)-1)+V0_HOST
a_host[i] = -DV1_HOST/2*np.sin(np.pi*(time[i]-T_HOST)*A1_HOST/DV1_HOST)*np.pi*A1_HOST/DV1_HOST
t2 = t1 + DV2_HOST / A2_HOST
i = np.logical_and(time > t1, time <= t2)
v_host[i] = -DV2_HOST/2*(np.cos(np.pi*(time[i]-t1)*A2_HOST/DV2_HOST)-1)+V0_HOST-DV1_HOST
a_host[i] = DV2_HOST/2*np.sin(np.pi*(time[i]-t1)*A2_HOST/DV2_HOST)*np.pi*A2_HOST/DV2_HOST
v_host[time > t2] = V0_HOST - DV1_HOST + DV2_HOST

plt.plot(time, v_host, label="host")
plt.plot(time, v_lead, label="lead")
plt.xlabel("Time [s]")
plt.ylabel("Speed [m/s]")
plt.legend()

In [ ]:
plt.plot(time, a_host, label="host")
plt.plot(time, a_lead, label="lead")
plt.xlabel("Time [s]")
plt.ylabel("Acceleration [m/s$^2$]")
plt.legend()

In [ ]:
distance = D_INIT + np.cumsum(v_lead - v_host)*0.01
plt.plot(time, distance)

In [ ]:
prob_ws = np.zeros_like(time)
for i, (vh, vl, gap) in enumerate(zip(v_host, v_lead, distance)):
    if vh > vl:
        ttc = gap / (vh - vl)
        prob_ws[i] = ws.prob_collision(ttc, vh-vl)

In [ ]:
prob_new = np.zeros_like(time)
for i, (vh, vl, al, gap) in enumerate(zip(v_host, v_lead, a_lead, distance)):
    prob_new[i] = prob_ngsim(vl, al, vh, gap)

In [ ]:
plt.plot(time, prob_ws, label="WS")
plt.plot(time, prob_new, label="New")
plt.xlabel("Time [s]")
plt.ylabel("Probability of collision")
plt.legend()

In [ ]:
len(time)

## Try method for scenario 3: Collision

In [ ]:
np.exp(4.12)